In [ ]:
import pandas as pd
import numpy as np
import os
import json
import requests

pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [ ]:
lct = "/Users/nam-yeong/Downloads/balso_tmp"
folders = os.listdir(lct)

for i in folders:
    print(os.path.join(lct,i))
    


Clicl에 상품정보 담기지 않은 개발 이슈. Impression > CLick 퍼널 이어진 데이터를 선별해서 상품 정보 Stitching

In [ ]:
tst = folders[1]
# chunksize = 5000
# nrows = 1000
df = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{tst}")
df.query("`Event Category`.str.contains('AD')")
conv_users = list(set(df.query("`Event Category`.str.contains('Click')")['Airbridge Device ID'].tolist()))
df = df.query("`Airbridge Device ID` in @conv_users")
df[(df['Event Category'].str.contains('Impression') )& (df['Event Datetime']>='2024-04-08T23:59:50+09:00')]

df.drop_duplicates(subset=['Airbridge Device ID','Event Datetime','Event Category','Product List'],inplace=True)


In [ ]:
len(df[df['Event Category'].str.contains('Click')])

In [ ]:
df.sort_values(by=['Airbridge Device ID', 'Event Datetime','Event Category'],ascending=[True,True,False], inplace=True)
df['prev_prop'] = df.groupby('Airbridge Device ID')['Product Name'].shift(1)
# df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]
df = df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]

df

In [ ]:
len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]) 

자정에 데이터 끊기는 이슈. 타겟 date, d+1 데이터 로드해서 처리 후, d+1 데이터는 burn \
마지막 데이터의 경우, Click에 상품정보 포함되어 있음. 따라서 burn 이후, 정상 데이터 merge